In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# check the status code to start as a habit

# header to say I am a browser and not some generic script
headers = {
    "User-Agent": "Mozilla/5.0"
}

url = 'https://www.nfl.com/stats/team-stats/'

def get_stats(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve the webpage")
        print(response.status_code)
        exit()
    soup = BeautifulSoup(response.content, 'html.parser')

    column_headers = soup.find("thead").text.splitlines()[2:]
    table_body = soup.find("tbody")
    rows = table_body.find_all("tr")
    #this was checking the frist team's data to make sure it grabs everything in the correct format
    # cells = rows[1].find_all("td")

    all_team_stats = []
    
    for row in rows:
        cells = row.find_all("td")
        #if there was a blank for or seperator row, this would skip it
        if not cells:
            continue

        #special case because table has formal team name and shortened name. Only want shortened name
        team_data = []
        team_cell = cells[0]
        short_team_name = team_cell.find("div", class_="d3-o-club-fullname")
        team_data.append(short_team_name.get_text(strip=True))

        #grab the rest of the stats
        for cell in cells[1:]:
            team_data.append(cell.get_text(strip=True))
        
        all_team_stats.append(team_data)
    
    return column_headers, all_team_stats

In [3]:
print(get_stats(url, headers))

(['', '', 'Team', 'Att', 'Cmp', 'Cmp %', 'Yds/Att', 'Pass Yds', 'TD', 'INT', 'Rate', '1st', '1st%', '20+', '40+', 'Lng', 'Sck', 'SckY', ''], [['Cardinals', '649', '427', '65.8', '6.7', '4354', '29', '11', '92.7', '216', '33.3', '49', '5', '50', '59', '399'], ['Bengals', '640', '412', '64.4', '6.6', '4244', '36', '17', '91', '213', '33.3', '47', '5', '64', '36', '239'], ['Cowboys', '624', '419', '67.2', '7.6', '4735', '31', '12', '98.2', '229', '36.7', '56', '12', '86', '31', '208'], ['Broncos', '613', '388', '63.3', '6.4', '3931', '25', '11', '87.7', '196', '32', '47', '7', '52T', '23', '124'], ['Rams', '598', '388', '64.9', '7.9', '4707', '46', '8', '109', '236', '39.5', '72', '8', '88T', '23', '150'], ['Saints', '591', '397', '67.2', '6.8', '4027', '19', '12', '88.7', '183', '31', '33', '6', '87', '49', '299'], ['Chiefs', '584', '363', '62.2', '6.8', '3947', '23', '12', '86.6', '189', '32.4', '52', '9', '61', '47', '308'], ['Lions', '582', '394', '67.7', '7.8', '4567', '35', '8', '10